In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import math
import re

In [2]:
def display(img, cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [3]:
def gammaCorrection(img, coeff):
    return img**(1./coeff)

In [4]:
training_img_dirs = os.listdir('data-set')
training_set_length = len(training_img_dirs)

In [5]:
trainingImages = np.zeros((training_set_length,200, 200))

In [6]:
vecsTrans = np.zeros((training_set_length, 40000))
tempVecs = np.zeros((1 , 40000))
for i in range(training_set_length):
    trainingImages[i] =  cv2.imread('data-set/'+training_img_dirs[i], 0)
    vecsTrans[i] = np.asarray(trainingImages[i]).reshape( 1 , 40000)
    tempVecs = tempVecs + ((1/training_set_length) * vecsTrans[i])

In [7]:
for i in range(training_set_length):
    vecsTrans[i] = vecsTrans[i] - tempVecs

In [8]:
vecs = vecsTrans.transpose()
# print (vecs.shape)

In [9]:
cov = np.dot(vecsTrans,vecs)
cov = cov/training_set_length

In [10]:
w, v = np.linalg.eig(cov)
newEigenVec = np.zeros((training_set_length, 40000))
for i in range(training_set_length):
    temp = np.dot(vecs, v[i])
    newEigenVec[i] = temp
    #newEigenVec[i] = temp*(1/np.linalg.norm(temp))             # notmalize the vector

In [11]:
# sortedEigenVect = [0]*training_set_length
# for i in range(v.shape[0]):
#     sortedEigenVect[i] = np.append(v[i], w[i])

In [12]:
# def sortLast(val):
#     return val[-1] 

In [13]:
# sortedEigenVect.sort(key=sortLast,reverse=True)

In [14]:
wArr = np.zeros((training_set_length,training_set_length))
for i in range(training_set_length):
    for j in range(training_set_length):
        wArr[i][j] = np.dot(newEigenVec[j] , vecsTrans[i].transpose())

In [15]:
maxDistanceArr = []
for i in range(training_set_length):
    for j in range (i+1 , training_set_length):
        maxDistanceArr.append(np.linalg.norm(wArr[i] - wArr[j]))
maxDistance = max(maxDistanceArr)/2
print (maxDistance)

608857715.6563683


In [16]:

detector = cv2.CascadeClassifier('lbpcascade_frontalface.xml')

In [17]:
def detect_face(img, text):
    face_img=img.copy()
    face_rects = detector.detectMultiScale(face_img)
    cropped_imgs=[]
    for (x,y,w,h) in face_rects:
        image = cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,255,255),10)
        if (text == 'not recognized'):
            cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        else:
            cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)    
        cropped_imgs.append(img[y:y+h,x:x+w])
    return face_img , cropped_imgs


In [18]:
def recognize_face(img):
    imgVec = np.asarray(img).reshape( 1 , 40000)
    phayNew = (imgVec - tempVecs).transpose()
    wNew = np.zeros((training_set_length))
    for i in range(training_set_length):
        wNew[i] = np.dot(newEigenVec[i] , phayNew)
    min = np.linalg.norm(wNew - wArr[0])  
    minIndex = 0
    for i in range (1 , training_set_length):
        result = np.linalg.norm(wNew - wArr[i])
        if (result < min):
            min = result
            minIndex = i  

    if (min <= maxDistance):  
        #print(min)     
        return minIndex     
    return -1       

In [19]:
# Function to prepare the name of the person by removing the image extention from the string
def face_name(str):
    pattern = r'[0-9]'
    new_string = re.sub(pattern, '', str)
    return new_string[:-4]

In [20]:
vid = cv2.VideoCapture(0)
text = 'not recognized'
while(True):
      

    ret, frame = vid.read(0)
    
    frame , imgs =detect_face(frame, text)
    cv2.imshow('frame', frame)
    if (len(imgs) > 0):
        resized_img = cv2.resize(imgs[0], (200, 200), interpolation = cv2.INTER_AREA)
        last =cv2.cvtColor(resized_img,cv2.COLOR_BGR2GRAY)
        index = recognize_face(last)
        if (index != -1):
            text = face_name(training_img_dirs[index])
            #text = training_img_dirs[index]
        else:
            text = "not recognized"    
    # press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# Destroy all the windows
cv2.destroyAllWindows()
# After the loop release the cap object
vid.release()

